# 13 用 Embeddings 實作 台灣旅遊客服機器人

## 13-2 實作 Embeddings

### 建立環境

安裝套件

In [ ]:
!pip install langchain
!pip install openai
!pip install python-dotenv

設定環境變數 OPENAI_API_KEY

In [ ]:
# 匯入函式
from dotenv import load_dotenv
load_dotenv()

建立 OpenAIEmbeddings 物件

In [ ]:
# 匯入類別
from langchain.embeddings import OpenAIEmbeddings
# 建立物件
embeddings = OpenAIEmbeddings()

安裝 tiktoken 模組

In [ ]:
!pip install tiktoken

### 文字轉向量

In [ ]:
doc = [
    "天空是藍色的",
    "天空不是紅色的",
    "sky is blue",
    "莓果是藍色的",
    "Betty 是一隻貓"
]
# embed_documentst 嵌入多個字串
embedded = embeddings.embed_documents(doc)
len(embedded), len(embedded[0])

len(embedding)的結果為5，表示有5個字串的嵌入向量。 len(embedding[0])的結果為1536，表示每個嵌入向量的維度是1536

In [ ]:
# embed_query 嵌入單個字串
embedded_query = embeddings.embed_query("天空的顏色是？")
len(embedded_query)

一個嵌入向量維度是1536

In [ ]:
embedded_query

### 比較關聯性 - 餘弦相似度

越接近1越相似
相反越接近-1越不相似

In [ ]:
from openai import embeddings_utils
for doc_res, doc in zip(embedded, doc):
    # 使用餘弦相似度計算
    similarity = embeddings_utils.cosine_similarity(
        embedded_query,doc_res)
    print(f'"{doc}" 與問題相似度：{similarity}')

得出最接近的是"天空是藍色的",最遠的是""Betty是一隻貓""

## 13-3 向量資料庫

### 動手操作

使用 !curl 下載本章範例文件

交通部觀光局原網址

https://www.taiwan.net.tw/userfiles/file/2023/%E5%8C%97%E5%8F%B0%E7%81%A3%E6%97%85%E9%81%8A%E7%B0%A1%E4%BB%8B_202212.pdf

https://www.taiwan.net.tw/userfiles/file/2023/%E4%B8%AD%E5%8F%B0%E7%81%A3%E6%97%85%E9%81%8A%E7%B0%A1%E4%BB%8B_202212.pdf

https://www.taiwan.net.tw/userfiles/file/2023/%E5%8D%97%E5%8F%B0%E7%81%A3%E6%97%85%E9%81%8A%E7%B0%A1%E4%BB%8B_202212.pdf

https://www.taiwan.net.tw/userfiles/file/2023/%E6%9D%B1%E5%8F%B0%E7%81%A3%E6%97%85%E9%81%8A%E7%B0%A1%E4%BB%8B_202212.pdf

In [ ]:
!curl "https://flagtech.github.io/F3762/api.srt" --output "api.srt"
!mkdir "旅遊"
!curl -L "https://ppt.cc/fTdnwx" -o "旅遊/nt.pdf"
!curl -L "https://ppt.cc/fZJ4Zx" -o "旅遊/ct.pdf"
!curl -L "https://ppt.cc/fV9Q9x" -o "旅遊/st.pdf"
!curl -L "https://ppt.cc/fKRvMx" -o "旅遊/et.pdf"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

安裝模組

In [ ]:
!pip install chromadb
!pip install pypdf
!pip install pysrt

建立語言模型

In [ ]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0)

資料匯入

In [ ]:
from langchain.document_loaders import SRTLoader
srt_loader = SRTLoader('/content/api.srt')
# 使用 load() 提取內容
doc=srt_loader.load()

In [ ]:
doc

資料分割

使用文字分割器分割100個 tokens 成一個 Document



In [ ]:
from langchain.text_splitter import (
        RecursiveCharacterTextSplitter)
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,chunk_overlap=10)
# 使用 split_documents 方法分割 Documents
docs = text_splitter.split_documents(doc)

In [ ]:
docs

將資料存入數據庫

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(documents=docs,
                    embedding=embeddings,
                    persist_directory='/content/drive/MyDrive/db')

打開左邊欄位可以看到 db 資料夾

儲存在本機端

In [ ]:
db.persist()

### 查詢資料

調用數據庫方式

In [ ]:
db_one=Chroma(persist_directory='/content/drive/MyDrive/db',
              embedding_function=embeddings)

查詢與輸入相似的第一筆資料

In [ ]:
# 以字串查詢
docs = db_one.similarity_search("temperature的豐富度是多少？", k=1,search_type=['cosine'])
print(docs[0].page_content)

利用向量查詢相似度接近的第一筆資料

In [ ]:
# 將問題變成向量去查詢
embedding_vector = embeddings.embed_query(
                "temperature的豐富度是多少？")
docs = db_one.similarity_search_by_vector(embedding_vector, k=1)
print(docs[0].page_content)

返回4筆資料向量的餘弦距離數值

In [ ]:
# 查詢相似度數值
docs = db_one.similarity_search_with_score(
                "temperature的豐富度是多少？", k=4)
for doc in docs:
    print(doc[0].page_content,doc[1])

## 13-4 台灣旅遊客服機器人

### 快速建立資料庫

建立 VectorstoreIndexCreator 物件

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index_creator = VectorstoreIndexCreator(
            embedding=embeddings,
            text_splitter=text_splitter,
            vectorstore_cls=Chroma,
            vectorstore_kwargs={
            "persist_directory":"/content/drive/MyDrive/vector"})

載入旅遊簡介資料

In [ ]:
# 匯入類別
from langchain.document_loaders import PyPDFLoader
import os

In [ ]:
# 匯入資料
loaders=[]
allFileList = os.listdir('/content/旅遊')
for file in allFileList:
    load=PyPDFLoader(f'/content/旅遊/{file}')
    loaders.append(load)

匯入資料並建立數據庫

In [ ]:
docsearch = index_creator.from_loaders(loaders)
docsearch = docsearch.vectorstore.persist()

### 創建 RetrievalQA

建立對話提示模板和 Memory

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "你是專業客服人員,請根據上下文來回答問題,"
        "你不知道答案就說你不知道, 不要試圖編造答案。\n"
        "---------\n"
        "{context}\n"
        "---------\n"
        "{history}"
        ),
    # context 和 question 為 RetrievalQA 的固定用法
    HumanMessagePromptTemplate.from_template(
        "使用繁體中文回答,對問題會盡力回答,"
        "回答問題後會再詢問是否還有什麼問題:\n"
        "Q:{question}")
])

串接對話提示模板和 Memory

In [ ]:
from langchain.chains import RetrievalQA

使用from_chain_type加入模板

In [ ]:
# 用字典存入參數
chain_type_kwargs = {"prompt":prompt,
                     "memory":ConversationSummaryBufferMemory(
                     llm=model,
                     max_token_limit=1500,
                     memory_key="history",
                     input_key="question")}

In [ ]:
db_two=Chroma(persist_directory='/content/drive/MyDrive/vector',
              embedding_function=embeddings)

In [ ]:
qa = RetrievalQA.from_chain_type(
            llm=model,
            # as_retriever() 方法讓資料庫變成檢索器
            retriever=db_two.as_retriever(search_kwargs={"k":3}),
            chain_type_kwargs=chain_type_kwargs)

In [ ]:
qa.run("嘉義檜意森活村?")

### 使用 Gradio 建立網頁程式

安裝gradio

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

儲存對話內容

In [ ]:
history=[]
def user(user_message):
        response = qa.run(user_message)
        #將對話紀錄儲存到串列中
        history.append((user_message,response))
        return history

建立網頁對話窗

In [ ]:
web_chat = gr.Interface(
    fn = user,
    inputs = [gr.Textbox(label="請輸入問題：")],
    outputs = [gr.Chatbot(label="對話記錄")])

啟動網頁程式

In [ ]:
web_chat.launch()

結束網頁程式

In [ ]:
web_chat.close()

分享程式

In [ ]:
web_chat.launch(share=True)